In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from statsmodels.stats.weightstats import DescrStatsW

# ==========================================
# 1. КОНФИГУРАЦИЯ И ЗАГРУЗКА
# ==========================================
# Настройки визуализации
sns.set_theme(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (12, 7)
PALETTE_GENDER = {0: "#2c3e50", 1: "#e74c3c"}  # Мужчины: Синий, Женщины: Красный


# Функция поиска файлов (Умный поиск)
def load_nhanes_data():
    print("🔄 Загрузка и объединение данных...")
    root = Path.cwd()
    # Пытаемся найти корень проекта, если мы в подпапке
    if root.name == "notebooks":
        root = root.parent

    # Список нужных файлов (по паттернам)
    files_needed = {
        "DEMO": "*DEMO_J.xpt",  # Демография
        "BMX": "*BMX_J.xpt",  # Тело (BMI)
        "LAB": "*PBCD_J.xpt",  # Металлы
        "DXA": "*DXX_J.xpt",  # Рентген (Жир/Мышцы)
        "DEP": "*DPQ_J.xpt",  # Депрессия (если она в отдельном файле, иначе берем из master)
    }

    dfs = {}
    for key, pattern in files_needed.items():
        found = list(root.rglob(pattern))
        if found:
            print(f"   ✅ Нашел {key}: {found[0].name}")
            dfs[key] = pd.read_sas(found[0])
        else:
            print(f"   ⚠️ Не нашел {key} (пропускаем)")

    if not dfs:
        raise ValueError("Не найдено ни одного файла!")

    # СБОРКА (MERGE) по SEQN
    # Начинаем с Демографии как основы
    base_df = dfs["DEMO"][
        ["SEQN", "RIAGENDR", "RIDAGEYR", "INDFMPIR", "WTMEC2YR"]
    ].copy()

    # Маппинг имен (согласно твоему конфигу)
    base_df = base_df.rename(
        columns={
            "RIAGENDR": "Gender",
            "RIDAGEYR": "Age",
            "INDFMPIR": "Poverty_Ratio",
            "WTMEC2YR": "MEC_Weight",
        }
    )

    # Кодируем пол: 1->0 (М), 2->1 (Ж)
    base_df["Gender"] = base_df["Gender"].replace({1: 0, 2: 1})

    # Присоединяем Металлы (Ртуть)
    if "LAB" in dfs:
        lab = dfs["LAB"][["SEQN", "LBXTHG"]].rename(
            columns={"LBXTHG": "Mercury_Total_ugL"}
        )
        base_df = base_df.merge(lab, on="SEQN", how="left")

    # Присоединяем BMI
    if "BMX" in dfs:
        bmx = dfs["BMX"][["SEQN", "BMXBMI", "BMXWAIST"]].rename(
            columns={"BMXBMI": "BMI", "BMXWAIST": "Waist_cm"}
        )
        base_df = base_df.merge(bmx, on="SEQN", how="left")

    # Присоединяем DXA (Жир и Мышцы)
    if "DXA" in dfs:
        dxa = dfs["DXA"][["SEQN", "DXDTOPF", "DXDTOLE"]].rename(
            columns={"DXDTOPF": "Body_Fat_Pct", "DXDTOLE": "Lean_Mass_g"}
        )
        base_df = base_df.merge(
            dxa, on="SEQN", how="left"
        )  # Left join, т.к. рентген есть не у всех

    # Присоединяем Депрессию (Таргет)
    # ПРИМЕЧАНИЕ: Здесь предполагается простая логика PHQ-9.
    # Если у тебя таргет уже посчитан в файле, используй его.
    # Для примера я генерирую таргет заглушку, если файла нет, НО ТЫ ДОЛЖЕН ИСПОЛЬЗОВАТЬ СВОЙ МЕТОД
    # ... (здесь код расчета Depression из DPQ_J, если нужно) ...
    # Предположим, что Depression уже есть или мы его считаем:
    if "DEP" in dfs:
        dep_cols = [c for c in dfs["DEP"].columns if c.startswith("DPQ")]
        # Простая сумма > 10 = Депрессия (Упрощенно)
        dfs["DEP"]["Score"] = dfs["DEP"][dep_cols].sum(axis=1)
        dfs["DEP"]["Depression"] = (dfs["DEP"]["Score"] >= 10).astype(int)
        base_df = base_df.merge(
            dfs["DEP"][["SEQN", "Depression"]], on="SEQN", how="inner"
        )

    return base_df


# Загружаем
df = load_nhanes_data()
df = df[df["MEC_Weight"] > 0]  # Убираем нулевые веса
print(f"📊 Итоговый датасет: {df.shape}")


# ==========================================
# 2. ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ==========================================
def get_weighted_stats(
    data, group_col, target_col="Depression", weight_col="MEC_Weight"
):
    """Считает взвешенную долю депрессии для групп"""
    stats = []
    for cat in sorted(data[group_col].dropna().unique()):
        grp = data[data[group_col] == cat]
        if len(grp) > 30:  # Игнорируем слишком малые группы
            w_mean = DescrStatsW(grp[target_col], weights=grp[weight_col]).mean
            stats.append({"Category": cat, "Depression_Rate": w_mean, "N": len(grp)})
    return pd.DataFrame(stats)


# ==========================================
# НАХОДКА 1: РТУТНЫЙ ПАРАДОКС (MERCURY)
# ==========================================
print("\n--- 1. Анализ Ртути и Дохода ---")
df["Log_Mercury"] = np.log10(df["Mercury_Total_ugL"] + 0.01)

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# График 1.1: Связь Ртуть-Депрессия (Boxplot)
sns.boxplot(
    x="Depression",
    y="Log_Mercury",
    data=df,
    ax=axes[0],
    palette="Set2",
    showfliers=False,
)
axes[0].set_title("Уровень ртути у здоровых (0) и больных (1)")
axes[0].set_ylabel("Log10 Mercury (ug/L)")

# График 1.2: Разгадка (Scatterplot Доход vs Ртуть)
sample = df.sample(2000, weights="MEC_Weight", random_state=42)  # Сэмпл для красоты
sns.regplot(
    x="Poverty_Ratio",
    y="Log_Mercury",
    data=sample,
    ax=axes[1],
    scatter_kws={"alpha": 0.3, "color": "grey"},
    line_kws={"color": "red"},
)
axes[1].set_title("Разгадка: Связь Дохода и Потребления Рыбы (Ртути)")
axes[1].set_xlabel("Poverty Ratio (Богаче ->)")

plt.tight_layout()
plt.show()

# ==========================================
# НАХОДКА 2: ИЛЛЮЗИЯ BMI (MEN VS WOMEN)
# ==========================================
print("\n--- 2. Иллюзия BMI (Сравнение полов) ---")

# Классификация BMI
df["BMI_Cat"] = pd.cut(
    df["BMI"],
    bins=[0, 18.5, 25, 30, 100],
    labels=["Underweight", "Normal", "Overweight", "Obese"],
)

# Считаем статистику отдельно для М и Ж
fig, axes = plt.subplots(1, 2, figsize=(18, 6), sharey=True)

for gender, ax, title in zip([0, 1], axes, ["Мужчины (Men)", "Женщины (Women)"]):
    subset = df[df["Gender"] == gender].copy()
    stats = get_weighted_stats(subset, "BMI_Cat")

    sns.barplot(
        x="Category",
        y="Depression_Rate",
        data=stats,
        ax=ax,
        palette="Blues_d",
        edgecolor="black",
    )

    ax.set_title(title, fontweight="bold")
    ax.set_xlabel("")
    if gender == 0:
        ax.set_ylabel("Вероятность Депрессии")

    # Подписи
    for i, row in stats.iterrows():
        ax.text(
            i,
            row["Depression_Rate"],
            f"{row['Depression_Rate']:.1%}",
            ha="center",
            va="bottom",
            fontweight="bold",
        )

plt.suptitle(
    "BMI Paradox: Почему мужчины с лишним весом (Overweight) счастливее?", y=1.05
)
plt.show()

# ==========================================
# НАХОДКА 3: ИСТИНА DXA (ЖИР И МЫШЦЫ)
# ==========================================
print("\n--- 3. Реальный состав тела (DXA Scan) ---")

# Фильтр: Только те, у кого есть DXA, и возраст 20-60 (активное население)
dxa_df = df[(df["Body_Fat_Pct"].notna()) & (df["Age"].between(20, 60))].copy()


# A. КЛАССИФИКАЦИЯ ПО ЖИРУ (РАЗНАЯ ДЛЯ М И Ж!)
def classify_fat_gender(row):
    pct = row["Body_Fat_Pct"]
    gen = row["Gender"]

    # Мужчины (0): Атлет <18%, Норма 18-25%, Ожирение >25%
    if gen == 0:
        if pct < 18:
            return "1. Low Fat / Athletic"
        elif pct < 25:
            return "2. Normal Fat"
        else:
            return "3. High Fat"
    # Женщины (1): Атлет <25%, Норма 25-32%, Ожирение >32%
    else:
        if pct < 25:
            return "1. Low Fat / Athletic"
        elif pct < 32:
            return "2. Normal Fat"
        else:
            return "3. High Fat"


dxa_df["Fat_Class"] = dxa_df.apply(classify_fat_gender, axis=1)

# B. КЛАССИФИКАЦИЯ ПО МЫШЦАМ (КВИНТИЛИ)
# Делим на 5 групп: от самых слабых (Low Muscle) до качков (High Muscle) внутри пола
dxa_df["Muscle_Quintile"] = dxa_df.groupby("Gender")["Lean_Mass_g"].transform(
    lambda x: pd.qcut(x, 5, labels=["1. Low Muscle", "2", "3", "4", "5. High Muscle"])
)

# ВИЗУАЛИЗАЦИЯ DXA
fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# Ряд 1: Влияние ЖИРА
for gender, ax, title in zip([0, 1], axes[0], ["Мужчины: % Жира", "Женщины: % Жира"]):
    subset = dxa_df[dxa_df["Gender"] == gender]
    stats = get_weighted_stats(subset, "Fat_Class")

    sns.barplot(
        x="Category",
        y="Depression_Rate",
        data=stats,
        ax=ax,
        palette="viridis",
        edgecolor="black",
    )
    ax.set_title(title, fontweight="bold")
    ax.set_ylim(0, 0.15)

    for i, row in stats.iterrows():
        ax.text(
            i,
            row["Depression_Rate"],
            f"{row['Depression_Rate']:.1%}\n(n={row['N']})",
            ha="center",
            va="bottom",
            fontweight="bold",
        )

# Ряд 2: Влияние МЫШЦ
for gender, ax, title in zip(
    [0, 1], axes[1], ["Мужчины: Мышечная Масса", "Женщины: Мышечная Масса"]
):
    subset = dxa_df[dxa_df["Gender"] == gender]
    stats = get_weighted_stats(subset, "Muscle_Quintile")

    # График с трендом
    sns.pointplot(
        x="Category",
        y="Depression_Rate",
        data=stats,
        ax=ax,
        color="#e74c3c" if gender == 1 else "#2c3e50",
        scale=1.5,
    )
    sns.barplot(
        x="Category",
        y="Depression_Rate",
        data=stats,
        ax=ax,
        color="lightgrey",
        alpha=0.5,
        edgecolor="black",
    )

    ax.set_title(title + " (Sarcopenia Risk)", fontweight="bold")
    ax.set_ylabel("Риск Депрессии") if gender == 0 else ax.set_ylabel("")
    ax.set_ylim(0, 0.15)

plt.tight_layout()
plt.show()